In [1]:
import ray
from ray.rllib.algorithms.ppo import PPO, PPOConfig
from ray import tune
import pprint

In [2]:
ray.shutdown()
ray.init(ignore_reinit_error = True)

2024-04-27 20:55:43,133	INFO worker.py:1749 -- Started a local Ray instance.


Python version:,3.11.5
Ray version:,2.12.0


In [3]:
pp = pprint.PrettyPrinter(depth = None, width = 200)

config = PPOConfig()
config_dict = config.to_dict()
pp.pprint(config_dict)

{'_AlgorithmConfig__prior_exploration_config': None,
 '_disable_action_flattening': False,
 '_disable_execution_plan_api': -1,
 '_disable_initialize_loss_from_dummy_batch': False,
 '_disable_preprocessor_api': False,
 '_enable_new_api_stack': False,
 '_enable_rl_module_api': -1,
 '_env_to_module_connector': None,
 '_fake_gpus': False,
 '_is_atari': None,
 '_learner_class': None,
 '_learner_connector': None,
 '_model_config_dict': {},
 '_module_to_env_connector': None,
 '_per_module_overrides': {},
 '_rl_module_spec': None,
 '_run_training_always_in_thread': False,
 '_tf_policy_handles_more_than_one_loss': False,
 'action_mask_key': 'action_mask',
 'action_space': None,
 'actions_in_input_normalized': False,
 'add_default_connectors_to_env_to_module_pipeline': True,
 'add_default_connectors_to_learner_pipeline': True,
 'add_default_connectors_to_module_to_env_pipeline': True,
 'algorithm_config_overrides_per_module': {},
 'always_attach_evaluation_results': True,
 'auto_wrap_old_gym_env

In [4]:
config['env'] = 'BipedalWalker-v3'

# We are setting the model to train via tensorflow
config['framework'] = 'tf'

In [5]:
config['lr'] = 5e-05

In [6]:
stop = {
    'timesteps_total': 250000
}

In [7]:
analysis = tune.run(
    'PPO',
    config = config,
    stop = stop,
    checkpoint_at_end = True,
    checkpoint_freq = 1000,
)

2024-04-27 20:56:02,088	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/Users/alexplash/anaconda3/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/Users/alexplash/anaconda3/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/alexplash/anaconda3/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


(PPO pid=63094) Trainable.setup took 15.931 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(PPO pid=63094) Install gputil for GPU system monitoring.
(PPO pid=63094) 2024-04-27 20:56:35,476	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_timesteps_total,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_BipedalWalker-v3_3adab_00000,252000,"{'ObsPreprocessorConnector_ms': 0.006119728088378906, 'StateBufferConnector_ms': 0.004445075988769531, 'ViewRequirementAgentConnector_ms': 0.11861729621887207}","{'num_env_steps_sampled': 252000, 'num_env_steps_trained': 252000, 'num_agent_steps_sampled': 252000, 'num_agent_steps_trained': 252000}",{},678.2,{},-37.8054,-93.2034,-272.201,2,67820,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 12.524674320140834, 'cur_kl_coeff': 0.5062499999999999, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 3.3799964708461356, 'policy_loss': -0.0341086944655305, 'vf_loss': 3.406981219102939, 'vf_explained_var': 0.2566301616289282, 'kl': 0.014071973798776618, 'entropy': 11.41126291726225, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 58125.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 252000, 'num_env_steps_trained': 252000, 'num_agent_steps_sampled': 252000, 'num_agent_steps_trained': 252000}",252000,252000,252000,4000,395.02,252000,4000,395.02,0,2,0,0,4000,"{'cpu_util_percent': 26.693333333333335, 'ram_util_percent': 62.680000000000014}",{},{},{},"{'mean_raw_obs_processing_ms': 0.2962341796706708, 'mean_inference_ms': 0.9979795068333521, 'mean_action_processing_ms': 0.1952538590297133, 'mean_env_wait_ms': 0.3935905520513404, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -37.80543074111144, 'episode_reward_min': -272.2014183453091, 'episode_reward_mean': -93.2033846267871, 'episode_len_mean': 678.2, 'episode_media': {}, 'episodes_this_iter': 2, 'episodes_timesteps_total': 67820, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-49.34461472789012, -48.68372135606529, -40.888214477828406, -111.1725548055352, -44.78405137420019, -109.55734942572502, -129.83228251712399, -106.84283522121919, -109.37463094562665, -110.6659164208801, -38.202839458334495, -107.55230839786057, -132.1100932731274, -121.01934004531863, -121.65136217843929, -106.66544314674475, -106.48982656791124, -110.53737394000466, -133.3622463918794, -128.48691687297386, -103.14561739230591, -109.79722223208037, -126.2489125257147, -110.56739755802975, -45.5108935200336, -45.16360647359183, -51.709247200138286, -108.14889314686818, -41.931938188068486, -112.50698269766445, -107.13883769350127, -103.87035525517601, -115.77835467432627, -117.61576298354815, -109.55061502788402, -39.4566132901069, -110.47364988147841, -132.6863165245192, -46.865318447202306, -105.46540259215733, -136.6536987534576, -45.23290894703313, -48.65583342564916, -59.0216904875929, -44.31085595682138, -43.74589045116623, -118.11041362082648, -114.33997741754787, -129.76464484349015, -45.621720003678185, -43.11279635486127, -226.27534340332718, -110.51492038701724, -104.26563396884936, -131.8870327002269, -47.7832500013213, -106.90834782281021, -109.9734789232798, -116.44278589757593, -114.21522308595542, -42.62320690124544, -50.25534256394082, -112.81470110831845, -42.87594693415867, -46.564535504661464, -116.93149316692974, -49.691149577184824, -38.450620935976545, -107.34578243715565, -37.80543074111144, -108.2977852633577, -119.39405348527305, -170.5108973302504, -115.860915

2024-04-27 21:07:04,099	INFO tune.py:1004 -- Wrote the latest version of all result files and experiment state to '/Users/alexplash/ray_results/PPO_2024-04-27_20-56-02' in 0.0216s.
(PPO pid=63094) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/alexplash/ray_results/PPO_2024-04-27_20-56-02/PPO_BipedalWalker-v3_3adab_00000_0_2024-04-27_20-56-02/checkpoint_000000)
2024-04-27 21:07:04,513	INFO tune.py:1036 -- Total run time: 662.43 seconds (661.87 seconds for the tuning loop).


In [8]:
trial = analysis.get_best_trial(metric = 'episode_reward_mean', mode = 'max')
best_checkpoint = analysis.get_best_checkpoint(trial, metric = 'episode_reward_mean', mode = 'max')
print(f"Found best checkpoint: {trial.experiment_tag}: {best_checkpoint}")


Found best checkpoint: 0: Checkpoint(filesystem=local, path=/Users/alexplash/ray_results/PPO_2024-04-27_20-56-02/PPO_BipedalWalker-v3_3adab_00000_0_2024-04-27_20-56-02/checkpoint_000000)


In [2]:
from ray.rllib.algorithms.ppo import PPO
from ray.rllib.algorithms.ppo import PPOConfig

config = PPOConfig().to_dict()
config['lr'] = 5e-05
config['env'] = "BipedalWalker-v3"

new_trainer = PPO(config=config)

checkpoint_path = '/Users/alexplash/ray_results/PPO_2024-04-27_20-56-02/PPO_BipedalWalker-v3_3adab_00000_0_2024-04-27_20-56-02/checkpoint_000000'
new_trainer.restore(checkpoint_path)

print(new_trainer)

/Users/alexplash/anaconda3/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:493: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/alexplash/anaconda3/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/alexplash/anaconda3/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be 

PPO


In [3]:
import gym

env = gym.make('BipedalWalker-v3', render_mode = 'human')
observation = env.reset()

while True:
    env.render()
    print(observation)
    try:
        action = new_trainer.compute_single_action(observation)
        observation, reward, done, emptyBool, info = env.step(action)
    except Exception as e:
        print(f"An error occurred: {e}")
        break

    if done:
        break

env.close()

(array([ 2.7468945e-03,  4.0504979e-06, -5.2817113e-04, -1.6000027e-02,
        9.2281088e-02,  1.2261221e-03,  8.6001706e-01,  4.2827646e-04,
        1.0000000e+00,  3.2652814e-02,  1.2260687e-03,  8.5364079e-01,
       -8.3428243e-04,  1.0000000e+00,  4.4081357e-01,  4.4581968e-01,
        4.6142232e-01,  4.8954970e-01,  5.3410226e-01,  6.0246044e-01,
        7.0914817e-01,  8.8593090e-01,  1.0000000e+00,  1.0000000e+00],
      dtype=float32), {})
An error occurred: 'tuple' object has no attribute 'float'


*** SIGTERM received at time=1714278725 ***
PC: @     0x7ff819e7272e  (unknown)  kevent
    @        0x13b2eac54  (unknown)  absl::lts_20230125::WriteFailureInfo()
    @        0x13b2ea9bb  (unknown)  absl::lts_20230125::AbslFailureSignalHandler()
    @     0x7ff819eda37d  (unknown)  _sigtramp
    @        0x1010dd3a8  (unknown)  _PyRuntime
    @        0x100d9d957  (unknown)  _PyEval_EvalFrameDefault
    @        0x100d5a23d  (unknown)  _PyEval_Vector
    @        0x100d59fda  (unknown)  PyEval_EvalCode
    @        0x100d5547b  (unknown)  builtin_exec
    @        0x100ca2a1f  (unknown)  cfunction_vectorcall_FASTCALL_KEYWORDS
    @        0x100d96ef7  (unknown)  _PyEval_EvalFrameDefault
    @        0x100c3d89f  (unknown)  _PyFunction_Vectorcall
    @        0x100e2efbe  (unknown)  pymain_run_module
    @        0x100e2ea80  (unknown)  Py_RunMain
    @        0x100bcc1f8  (unknown)  main
    @     0x7ff819b243a6  (unknown)  start
[2024-04-27 21:32:05,460 E 63409 8814846] logging.cc:3

: 